In [1]:
import numpy as np
from collections import defaultdict
import pandas as pd
import ast

In [2]:
class HMM:
    def __init__(self, states, observations):
        self.states = states
        self.observations = observations
        self.start_probabilities = np.zeros(len(states))
        self.transition_probabilities = np.zeros((len(states), len(states)))
        self.emission_probabilities = np.zeros((len(states), len(observations)))

        self.state_index = {state: i for i, state in enumerate(states)}
        self.observation_index = {obs: i for i, obs in enumerate(observations)}

    def compute_start_probabilities(self, sequences):
        for sequence in sequences:
            start_state = sequence[0][1]  # The tag of the first token
            self.start_probabilities[self.state_index[start_state]] += 1
        self.start_probabilities /= np.sum(self.start_probabilities)

    def compute_transition_probabilities(self, sequences):
        for sequence in sequences:
            for i in range(len(sequence) - 1):
                current_state = sequence[i][1]
                next_state = sequence[i + 1][1]
                self.transition_probabilities[self.state_index[current_state], self.state_index[next_state]] += 1

        for i in range(len(self.states)):
            if(np.sum(self.transition_probabilities[i])==0):
                continue
            self.transition_probabilities[i] /= np.sum(self.transition_probabilities[i])

    def compute_emission_probabilities(self, sequences):
        for sequence in sequences:
            for token, state in sequence:
                self.emission_probabilities[self.state_index[state], self.observation_index[token]] += 1

        for i in range(len(self.states)):
            if(np.sum(self.emission_probabilities[i])==0):
                continue
            self.emission_probabilities[i] /= np.sum(self.emission_probabilities[i])

    def viterbi_algorithm(self, obs):
        viterbi_table = [[0.0 for _ in range(len(self.states))] for _ in range(len(obs))]
        backpointer = [[0 for _ in range(len(self.states))] for _ in range(len(obs))]

        for t in range(len(obs)):
            for s in range(len(self.states)):
                if t == 0:
                    viterbi_table[t][s] = self.start_probabilities[s] * self.emission_probabilities[s][obs[t]]
                else:
                    max_prob = -1
                    max_backpointer = -1

                    for s_prime in range(len(self.states)):
                        prob = viterbi_table[t-1][s_prime] * self.transition_probabilities[s_prime][s] * self.emission_probabilities[s][obs[t]]
                        if prob > max_prob:
                            max_prob = prob
                            max_backpointer = s_prime

                    viterbi_table[t][s] = max_prob
                    backpointer[t][s] = max_backpointer

        best_path_prob = max(viterbi_table[-1])
        best_path_pointer = max(range(len(self.states)), key=lambda s: viterbi_table[-1][s])
        best_path = [best_path_pointer]
        for t in range(len(obs)-1, 0, -1):
            best_path.insert(0, backpointer[t][best_path[0]])

        return best_path

In [3]:
def threshold_data(sequences, threshold):
    word_counts = defaultdict(int)
    for sequence in sequences:
        for token, state in sequence:
            word_counts[token] += 1

    unk_sequences = []
    for sequence in sequences:
        unk_sequence = [(token if word_counts[token] > threshold else 'UNK', state) for token, state in sequence]
        unk_sequences.append(unk_sequence)

    return unk_sequences

In [4]:
train = pd.read_csv('./telugu_train_data.csv')
test = pd.read_csv('./telugu_test_data.csv')

In [5]:
df_train = pd.read_csv('./telugu_train_data.csv', converters={'tokens': ast.literal_eval, 'ner_tags': ast.literal_eval})

In [6]:
train_data = [(row['tokens'], row['ner_tags']) for _, row in df_train.iterrows()]
unique_tags = set(tag for _, tags in train_data for tag in tags)
states = list(unique_tags)

In [7]:
train_hmm_data = [list(zip(tokens, tags)) for tokens, tags in train_data]

In [8]:
unk_sequences = threshold_data(train_hmm_data, 5)

In [9]:
print(unk_sequences[0])

[('స్థాపించబడిన', 3), ('సాఫ్ట్వేర్', 0)]


In [10]:
unique_tokens = set()
for sequence in unk_sequences:
    for token, _ in sequence:
        unique_tokens.add(token)

unique_tokens_list = list(unique_tokens)
num_observations = len(unique_tokens)

In [11]:
def train_hmm(hmm_model, sequences):
    hmm_model.compute_start_probabilities(sequences)
    hmm_model.compute_transition_probabilities(sequences)
    hmm_model.compute_emission_probabilities(sequences)

In [12]:
hmm_model = HMM(states, unique_tokens_list)

train_hmm(hmm_model, unk_sequences)

print("Start Probabilities:")
print(hmm_model.start_probabilities)
print("Transition Probabilities:")
print(hmm_model.transition_probabilities)
print("Emission Probabilities:")
print(hmm_model.emission_probabilities)

Start Probabilities:
[0.66402556 0.11883815 0.         0.11047168 0.         0.10666462
 0.        ]
Transition Probabilities:
[[8.41370570e-01 8.06962244e-02 0.00000000e+00 3.72564422e-02
  0.00000000e+00 4.06767635e-02 0.00000000e+00]
 [3.47151738e-01 1.29405527e-02 6.33020841e-01 3.04723506e-03
  2.68016850e-04 3.51917777e-03 5.24380794e-05]
 [7.06447729e-01 1.72742200e-02 2.57099070e-01 8.99470899e-03
  1.44499179e-03 8.52764094e-03 2.11640212e-04]
 [5.43750488e-01 4.31287881e-03 1.21218427e-03 8.17834196e-03
  4.39929246e-01 2.49720365e-03 1.19657675e-04]
 [3.99908806e-01 6.24505792e-03 1.37945364e-03 1.54221763e-02
  5.71064950e-01 5.84680561e-03 1.32750769e-04]
 [8.02106899e-01 5.99001005e-03 7.97017866e-04 1.46829503e-02
  2.25244180e-03 3.33014856e-02 1.40869195e-01]
 [5.87684336e-01 5.05679554e-03 9.21681945e-04 7.09944201e-03
  1.84336389e-03 3.41520526e-02 3.63242328e-01]]
Emission Probabilities:
[[1.80569578e-06 1.19691835e-04 1.03182616e-06 ... 2.57956541e-07
  3.61139157

In [13]:
test_tokens = test['tokens']
test_ner_tags = test['ner_tags']
misvals = []

N_test = len(test_tokens)

def compute_accuracy(hmm_model):
    count_correct_new = 0
    tot_new = 0
    correct = 0
    total = 0
    for idx in range(N_test):
        tags = eval(test_ner_tags[idx])
        observations = [obs for obs in eval(test_tokens[idx])]
        if(len(observations)==0):
            continue
        observed_tokens = [tok for tok in eval(test_ner_tags[idx])]
        observation_indices = [hmm_model.observation_index[obs] if obs in hmm_model.observation_index else hmm_model.observation_index['UNK'] for obs in observations]
        predicted_tags = hmm_model.viterbi_algorithm(observation_indices)
        for i,obs in enumerate(observations):
            if obs not in hmm_model.observation_index:
                tot_new += 1 
                if(predicted_tags[i] == tags[i]):
                    count_correct_new += 1

        for i in range(len(predicted_tags)):
            if(predicted_tags[i] == observed_tokens[i]):
                correct += 1
        total += len(observations)
    return correct / total, count_correct_new / tot_new


accuracy, unk_accuracy = compute_accuracy(hmm_model)
print("Accuracy:", accuracy)
print("UNK Accuracy:", unk_accuracy)

Accuracy: 0.9042406084351233
UNK Accuracy: 0.7178051511758119
